# Fantasy football draft 2019 Xmas Review

Here's some script to create interactive bokeh plots of scores based on draft picks from the 2019-20 Dixon Draft league. The data is currently up to end of GW17 - it's easy to update this on request. I've hosted the data on my website for now. There's a separate back end script that takes the draft picks and matches it up to the scores from the FPL API (this does lots of boring stuff like differentiating between Harvey and Ashley Barnes both with the web_name 'Barnes' FFS). Run the script sections below and have a play with the resulting graphs. Merry Christmas,
Jannik

## Import packages

In [78]:
# Import necessary packages
import yaml
import pandas as pd

# bokeh package elements below for visualisations
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Select, HoverTool, TapTool, CategoricalColorMapper, Label, BoxSelectTool
from bokeh.io import show, output_notebook
from bokeh.layouts import layout, column, row
from bokeh.models.widgets import Panel, Dropdown
from bokeh.themes import Theme

## Import data

In [79]:
draftees = pd.read_csv('http://www.jannikgiesekam.co.uk/draft_picks.csv') # read in results that have already been processed

In [80]:
draftees = draftees.sort_values('Draft_pick') # sort players by overall draft pick
draftees.Draft_pick = draftees.Draft_pick.astype(str)
draftees.Draft_round = draftees.Draft_round.astype(str)

# Add points per 90 minute metric
draftees['ppm'] = 90*draftees['Points']/draftees['Minutes']

## Create interactive plots

In [81]:
# Set custom hover tool tips using html
TOOLTIPS = """
    <div style="font-size:9px; margin:5px; width:300px" >
        <div>
            <span style="font-size: 12px; font-weight: bold;">@FN @SN</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold;">@Points points at @ppm{0.0} points per 90 mins</span>
        </div>
        <div>
            <span style="font-size: 11px">Pick number @Pick{0} in round @Round{0}</span>
        </div>
        <div>
            <span style="font-size: 11px">@Goals goals - @Assists assists - @CS clean sheets 
            in @Minutes minutes</span>
        </div> 
        <div>
            <span style="font-size: 11px">Plays for @Manager in @Team </span>
        </div>                       
"""
hover = HoverTool(tooltips=TOOLTIPS)

# Set tools
toolset = [hover,'box_select','tap','pan','wheel_zoom','box_zoom','save','reset']

output_notebook()

# Create lists for dropdowns and append All options
managerlist = draftees.Manager.unique().tolist()
managerlist.sort()
managerlist.insert(0,'All')
teamlist = draftees.Team.unique().tolist()
teamlist.sort()
teamlist.insert(0,'All')
draftroundlist = draftees.Draft_round.unique().tolist()
draftroundlist.sort()
draftroundlist.insert(0,'All')

# Create control lists
manager = Select(title="Manager", value="All", options=managerlist)
team = Select(title="Team", value="All", options=teamlist)

# Create list of all selectable options
uberlist = managerlist + teamlist
my_set = set(uberlist)
uberlist = list(my_set)
uberlist.sort()

# Set colour mapping by manager
mapper = CategoricalColorMapper(factors = managerlist, 
                                palette = ['#FFF000','#ad494a','#d6616b',
                                           '#e6550d','#3182bd','#31a354',
                                           '#ffc0cb','#009eff','#b98661',
                                           '#9e00ff','#9eff00','#00aedb', '#ffa500']) # Assign colours to each manager

# Map selection choices to df columns
axis_map = {
    "Points": "Points",
    "Draft Pick": "Draft_pick",
    "Goals Scored": "Goals",
    "Goals Conceded": "Goals_conceded",
    "Assists": "Assists",
    "Clean Sheets": "Clean_sheets",
    "Draft Round": "Draft_round",
    "Minutes": "Minutes",
    "Points per minute":"ppm"
}

# Create controls
x_axis = Select(title="X Axis", options=sorted(axis_map.keys()), value="Draft Pick")
y_axis = Select(title="Y Axis", options=sorted(axis_map.keys()), value="Points")
subset = Select(title="Show only", value="All", options=uberlist)

# Set plot dimensions, data source and glyphs
peda = figure(plot_height=700, plot_width=700, title="Dixon Draft 2019-20", tools=toolset)
source = ColumnDataSource(data=dict(x=[], y=[]))
peda.circle(x="x", y="y", source=source, size=13, color={'field':'Manager','transform':mapper}, line_color=None)
peda.x_range.start = 0
peda.y_range.start = 0

# Set initial plot data
df = draftees
source.data = dict(
    x=df['Draft_pick'],
    y=df['Points'],
    Manager=df['Manager'],
    Team=df['Team'],
    Player=df['Player'],
    Minutes=df['Minutes'],
    Goals =df['Goals'],
    Assists = df['Assists'],
    Conceded = df['Goals_conceded'],
    CS = df['Clean_sheets'],
    Points = df['Points'],
    Pick = df['Draft_pick'],
    Round = df['Draft_round'],
    FN = df['First_name'],
    SN = df['Second_name'],
    ppm = df['ppm']
)
peda.xaxis.axis_label = 'Draft Pick'
peda.yaxis.axis_label = 'Points'

# Create function to select companies based on subset selection
def select_targets():
    subset_val = subset.value
    selected = draftees
    if (subset_val != "All"):
        mask = (selected['Manager'].str.startswith(subset_val)|selected['Team'].str.startswith(subset_val))
        selected = selected.loc[mask]       
    return selected  
                          
def modify_doc(doc):
    
    def update():
        df = select_targets()
        
        # Update axis labels
        x_name = axis_map[x_axis.value]
        y_name = axis_map[y_axis.value]
        peda.xaxis.axis_label = x_axis.value
        peda.yaxis.axis_label = y_axis.value
        
        # Update source data to new values
        source.data = dict(
            x=df[x_name],
            y=df[y_name],
            Team=df['Team'],
            Points=df['Points'],
            Manager=df['Manager'],
            Player=df['Player'],
            Minutes=df['Minutes'],
            Goals =df['Goals'],
            Assists = df['Assists'],
            Conceded = df['Goals_conceded'],
            CS = df['Clean_sheets'],
            Pick = df['Draft_pick'],
            Round = df['Draft_round'],
            FN = df['First_name'],
            SN = df['Second_name'],
            ppm=df['ppm']
            )
    
    # Update plot when values for controls are changed
    controls = [x_axis, y_axis, subset]
    for control in controls:
        control.on_change('value', lambda attr, old, new: update())    
    
    # Arrange layout of plot
    sizing_mode = 'fixed'
    inputs = column(*controls, sizing_mode=sizing_mode)
    l = layout([
        [peda, inputs],
    ], sizing_mode=sizing_mode)
        
    doc.add_root(l)    
    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                background_fill_color: "#DDDDDD"
                outline_line_color: white
                toolbar_location: above
            Grid:
                grid_line_color: white
    """))
show(modify_doc)

Loading BokehJS ...

C:\Users\Jannik Giesekam\Anaconda3\lib\site-packages\ipykernel_launcher.py:165: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
